**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [1]:
#with li's idea with threshold , we can got 3.670 now
#without li's idea, we can got 3.675

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [2]:
a = [i for i in range(10)]
a[0:int(10 * 0.2 + 1)]

[0, 1, 2]

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
# import os
# os.listdir('../input')

['elo-merchant-category-recommendation', 'elomerchantfeatures']

In [4]:
df_merchant = pd.read_csv('../../ELO/merchants.csv')

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_hist_trans = pd.read_csv('../../ELO/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../../ELO/new_merchant_transactions.csv')

In [6]:
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [7]:
len(df_test) / len(df_train)  * 24000 

14693.91878841306

In [8]:
# for df in [df_new_merchant_trans,df_hist_trans]:
#     df['purchase_amount'].clip(upper = 1.5, inplace = True)
    

In [9]:
#使用countVectorizer对category特征进行处理，别人号称可以提升 千分之3
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(token_pattern='\w{1,}') 

**Purchasing Journeys**

[An Introduction to Predictive Customer Lifetime Value Modeling](https://www.datascience.com/blog/intro-to-predictive-modeling-for-customer-lifetime-value)

![](https://d2mxuefqeaa7sj.cloudfront.net/s_60ECB163AE4078E84C49514ED5D5B38C222B9C0EA718BEA3759DABF297034DEB_1488169955068_file.png)

In [10]:
hist_monthly_perchase_cnts = df_hist_trans.groupby('card_id').month_lag.value_counts().unstack()
hist_monthly_perchase_cnts.fillna(0, inplace=True)
hist_monthly_perchase_cnts.columns = [f'hist_per_cnts_month_lag{col}' for col in hist_monthly_perchase_cnts.columns]
new_merchant_monthly_perchase_cnts = df_new_merchant_trans.groupby('card_id').month_lag.value_counts().unstack()
new_merchant_monthly_perchase_cnts.fillna(0, inplace=True)
new_merchant_monthly_perchase_cnts.columns = [f'new_merchant_per_cnts_month_lag{col}' for col in new_merchant_monthly_perchase_cnts.columns]
df_train = df_train.join(hist_monthly_perchase_cnts, on='card_id')
df_train = df_train.join(new_merchant_monthly_perchase_cnts, on='card_id')
df_test = df_test.join(hist_monthly_perchase_cnts, on='card_id')
df_test = df_test.join(new_merchant_monthly_perchase_cnts, on='card_id')

In [11]:
# 结果会从 3.675 变成 3.679 ，so sad
# i = 0
# aggs = {}
# aggs['purchase_amount'] = ['mean']
# for df in [df_hist_trans,df_new_merchant_trans]:
#     df['purchase_amount'] = df['purchase_amount'].astype(np.float32)
#     df_month_purchase_cnts = df.groupby(['card_id','month_lag']).agg(aggs).unstack()
#     df_month_purchase_cnts.fillna(0, inplace = True)
#     df_train = df_train.join(df_month_purchase_cnts, on = 'card_id')
#     df_test = df_test.join(df_month_purchase_cnts, on = 'card_id')
#     if i == 0:
#         prefix = 'hist_'
#     else:
#         prefix = 'new_merchant_'
#     i += 1
#     df_train.columns = [prefix + c[0] + c[1] + str(c[2]) if type(c) is tuple else c for c in df_train.columns ]
#     df_test.columns = [prefix + c[0] + c[1] + str(c[2]) if type(c) is tuple else c for c in df_test.columns ]
#     del df_month_purchase_cnts
#     gc.collect()
#     time.sleep(5)

In [12]:
df_hist_trans.purchase_date=pd.to_datetime(df_hist_trans.purchase_date)
pur_date=df_hist_trans[df_hist_trans.month_lag==0].groupby('card_id').purchase_date.max()
card_id_nan_unique=df_hist_trans[df_hist_trans.card_id.isin(pur_date.index)==False].card_id.unique()
df=pd.DataFrame(card_id_nan_unique)
df['month_lag_date']=pd.to_datetime('2018-02') # Seetting all nan values to 2018 Feb
df.set_index(0,inplace=True)
new_map=df.month_lag_date
method=pur_date.append(new_map)
del df,pur_date
gc.collect()
time.sleep(5)

In [13]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    #修改特征
    df['category_2'].fillna(-1,inplace=True)
    df['category_3'].fillna('other',inplace=True)
    df['merchant_id'].fillna('other',inplace=True)
#     df.loc[df['installments'].isin([999,-1]),'installments'] = 0 

In [14]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [15]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    ##Think about this part!
    ## This line of code may be missleading since there is no single ref date. 
    df['month_diff'] = ((datetime.datetime(2018,6,1) - df['purchase_date']).dt.days)//30 # original code
    df['month_diff'] += df['month_lag']
    
    ## removing 'month_diff' reduces LB score to 3.682
    ## This line of code can be also missleading since for new merchant hist it might be negative. But why we care about it.
    #df['month_diff'] = ((df['card_id'].map(method) - df['purchase_date']).dt.days)//30 # method that I used in my code
    
    ##But we already have mongth lag for each purchase! So we don't really need it!
    ### month_diff seems to be important feature think more about it
    
    # This makes it even more irrelevant for my case since it will be equal to zero in most of the cases.So, let's remove it for my case
    #df['month_diff'] += df['month_lag']

In [16]:
(pd.to_datetime('2018-12-30')-pd.to_datetime('2019-12-30')).days//30

-13

In [17]:
#因为根据auth_flag 将特征分成了两部分，这里聚合一个全局的auth_flag 特征
aggs = {}
aggs['purchase_amount'] = ['sum']
# aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['authorized_flag'] = ['sum', 'mean','std']
aggs['card_id'] = ['size']
auth_flag = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_flag.columns = get_new_columns('auth_flag',aggs)
auth_flag.reset_index(inplace=True)
df_train = df_train.merge(auth_flag,on='card_id',how='left')
df_test = df_test.merge(auth_flag,on='card_id',how='left')
del auth_flag
gc.collect()

49

In [18]:
#对 authorized_flag进行结果编码
aggs = {}
for col in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
    df_hist_trans[col+'_auth_mean'] = df_hist_trans.groupby([col])['authorized_flag'].transform('mean')
    df_hist_trans[col+'_auth_sum'] = df_hist_trans.groupby([col])['authorized_flag'].transform('sum') 
    aggs[col+'_auth_mean'] = ['mean']
    aggs[col+'_auth_sum'] = ['sum'] 
auth_encoder = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_encoder.columns = get_new_columns('auth_encoder',aggs)
auth_encoder.reset_index(inplace=True)
df_train = df_train.merge(auth_encoder,on='card_id',how='left')
df_test = df_test.merge(auth_encoder,on='card_id',how='left')
del auth_encoder
gc.collect()

35

In [19]:
def aggregate_per_month(prefix,history,agg_func):
    grouped = history.groupby(['card_id', 'month_lag'])
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = [prefix + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = [prefix + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True) 
    return final_group

In [20]:
#对授权码 进行按月聚合
agg_func = {'authorized_flag': [ 'sum', 'mean','median']}
final_group =  aggregate_per_month('agg_per_month_total',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group

In [21]:
authorized_transactions = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [22]:
df_hist_trans.shape

(2516909, 33)

In [23]:
agg_func = {'purchase_amount': [ 'sum', 'mean', 'min', 'max', 'std']}
final_group =  aggregate_per_month('agg_per_month_auth',authorized_transactions,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist_new',df_new_merchant_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
gc.collect()

92

In [24]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [25]:
i = 0
for df in [authorized_transactions,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
    aggs['installments'] = ['sum','max','min','mean','var','median']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','median']
    
    ## I also remove this part since it is also irrelevant since it is equivelent to month_lag. But it makes it worser
    aggs['month_diff'] = ['mean','median']
#     aggs['authorized_flag'] = ['sum', 'mean','median']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    #产生交叉特征，内存有问题
    features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#     for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#             df[coli + colj] = df[coli].astype(str) + df[colj].astype(str)
#             features.append(coli + colj)
    for col in features:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum') 
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum'] 
#         添加特征，使用outlier进行编码
#         outliers_mean = df.groupby([col])['outliers'].mean()
#         outliers_sum = df.groupby([col])['outliers'].sum()
#         df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#         df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#         aggs[col+'_outliers_mean'] = ['mean']
#         aggs[col+'_outliers_sum'] =['sum']   
        
    if i == 0:
        prefix = 'auth_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    # df_hist_trans.sort_values(['card_id','purchase_date'],inplace = True)

    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_hist_trans_group[prefix + '_purchase_date_diff'] = (df_hist_trans_group[prefix + '_purchase_date_max'] - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_average'] = df_hist_trans_group[prefix + '_purchase_date_diff']/df_hist_trans_group[prefix + '_card_id_size']
    ###Think about below line of code  
    ###up to now for history means month_lag_0 to purdate_min
#     df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    ### This is new feature but I change the name to the old name since it makes more sense. Total number of days from first purchase to month lag 0 (until promotion)
    df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    ### This is old feature but I change the name to [prefix + '_first_purchase'] from [prefix + '_purchase_date_uptonow']
    ### since it makes more sense. Number of days for the last purchase from month_lag_0
    df_hist_trans_group[prefix + '_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    #下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
#     df_hist_trans_group[prefix + '_purchase_date_uptonow_ave'] =  df_hist_trans_group[prefix + '_purchase_date_uptonow']/df_hist_trans_group[prefix + '_purchase_date_average']
    #每一个card中未授权消费次数
#     df_hist_trans_group[prefix + '_unauthorized_number'] = df_hist_trans_group[prefix + '_card_id_size'] - df_hist_trans_group[prefix + '_authorized_flag_sum']
    #最近活跃时间，确实是一个强特征，感觉可以再挖出来几个特征，比如最近5次消费时间，最近10次消费时间，如果值比较小，说明最近很活跃
    #没有效果
    # grouped =  df_hist_trans.groupby('card_id')['purchase_date']
    # df_hist_trans_group['hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(5)).dt.days
    # df_hist_trans_group['hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(3)).dt.days
    # df_hist_trans_group['hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(10)).dt.days
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    gc.collect()
    time.sleep(5)
    #内存问题，暂时先放弃
#     for col in ['subsector_id','merchant_category_id','state_id','city_id']:
#         df[col]=df[col].astype(str)
#         grouped = df[['card_id',col]].groupby('card_id')[col].aggregate(lambda x: ' '.join(x))
#         cv_fit=cv.fit_transform(grouped)
#         cv_df = pd.DataFrame(cv_fit.toarray())
#         print(i,col)
#         if i == 1:
#             cv_df.columns = ['auth' + '_' + col + "_" + str(c) for c in cv_df.columns]
#         elif i == 2:
#              cv_df.columns = ['hist' + '_' + col + "_" + str(c) for c in cv_df.columns]
#         cv_df['card_id'] = grouped.index
#         df_train = df_train.merge(cv_df,on='card_id',how='left')
#         df_test = df_test.merge(cv_df,on='card_id',how='left')
#         del cv_df,grouped
#         gc.collect()
    del df
    gc.collect()
    time.sleep(5)

In [26]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_per_cnts_month_lag-13,hist_per_cnts_month_lag-12,hist_per_cnts_month_lag-11,hist_per_cnts_month_lag-10,...,hist_merchant_category_id_mean_mean,hist_merchant_category_id_sum_sum,hist_city_id_mean_mean,hist_city_id_sum_sum,hist_merchant_id_mean_mean,hist_merchant_id_sum_sum,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_first_purchase
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0.0,0.0,0.0,0.0,...,13.846728,1.144478e+07,17.928541,7.615314e+07,12.858409,337518.250000,164.0,12.615385,224.0,59.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0.0,21.0,22.0,13.0,...,2.988992,2.014900e+06,16.378971,6.238182e+07,1.745633,8938.264648,299.0,27.181818,334.0,34.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,6.0,6.0,5.0,7.0,...,1.316483,1.042644e+06,7.287305,5.252981e+05,-0.559304,-1012.340759,56.0,28.000000,368.0,312.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0.0,0.0,0.0,0.0,...,3.685194,2.026175e+06,2.260627,8.374476e+05,0.144859,1023.051819,37.0,7.400000,38.0,1.0


In [27]:
aggs = {}
#添加特征
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
            'state_id','city_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','median']

### Now we also need to remove from here since we already have month_lag
aggs['month_diff'] = ['mean','median']


aggs['weekend'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean','median']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

#添加特征   
features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#产生交叉特征，内存有问题
# for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#     for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         df_new_merchant_trans[coli + colj] = df_new_merchant_trans[coli].astype(str) + df_new_merchant_trans[colj].astype(str)
#         features.append(coli + colj)
for col in features:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    df_new_merchant_trans[col+'_sum'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']
    aggs[col+'_sum'] = ['sum']
    #添加特征，使用outlier进行编码
#     outliers_mean = df.groupby([col])['outliers'].mean()
#     outliers_sum = df.groupby([col])['outliers'].sum()
#     df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#     df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#     aggs[col+'_outliers_mean'] = ['mean']
#     aggs[col+'_outliers_sum'] =['sum']   
    
new_columns = get_new_columns('new_hist',aggs)
# df_new_merchant_trans.sort_values(['card_id','purchase_date'],inplace = True)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']


# it is fine for new mech transaction history. This means total number of days from month_lag_0 ti kast purchase
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_max']).dt.days

# new feature for the first purchase.This means total number of days from month_lag_0 to first purchase.
df_hist_trans_group['new_hist_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_min']).dt.days

#下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
# df_hist_trans_group['new_hist_purchase_date_uptonow_ave'] =  df_hist_trans_group['new_hist_purchase_date_uptonow']/df_hist_trans_group['new_hist_purchase_date_average']

#每一个card中未授权消费次数
df_hist_trans_group['new_hist_unauthorized_number'] = df_hist_trans_group['new_hist_card_id_size'] - df_hist_trans_group['new_hist_authorized_flag_sum']
# grouped = df_new_merchant_trans.groupby('card_id')['purchase_date']
# df_hist_trans_group['new_hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(5)).dt.days
# df_hist_trans_group['new_hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(3)).dt.days
# df_hist_trans_group['new_hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(10)).dt.days

df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()
del df_new_merchant_trans
gc.collect()
time.sleep(5)

In [28]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (df['card_id'].map(method) - df['first_active_month']).dt.days
    # 3.678，so bad
#     # add some interaction features
#     df['feature_1_hist_month_lag_max'] = df['hist_month_lag_max'] * df['feature_1']
#     df['feature_2_hist_month_lag_max'] = df['hist_month_lag_max'] * df['feature_2']
#     df['feature_3_hist_month_lag_max'] = df['hist_month_lag_max'] * df['feature_3']
#     df['feature_1_auth_hist_month_lag_max'] = df['auth_hist_month_lag_max'] * df['feature_1']
#     df['feature_2_auth_hist_month_lag_max'] = df['auth_hist_month_lag_max'] * df['feature_2']
#     df['feature_3_auth_hist_month_lag_max'] = df['auth_hist_month_lag_max'] * df['feature_3']
    
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #添加特征
    df['auth_hist_first_buy'] = (df['auth_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #修改特征
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min','auth_hist_purchase_date_max','auth_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    for f in ['auth_encoder_category_2_auth_sum_sum','auth_encoder_category_3_auth_sum_sum',
            'auth_encoder_state_id_auth_sum_sum','auth_encoder_subsector_id_auth_sum_sum',
            'auth_encoder_merchant_category_id_auth_sum_sum','auth_encoder_city_id_auth_sum_sum']:
        df[f] = df[f].astype(np.int64) * 1e-9
    #上面auth_flag已经聚合过了card_id_size ,purchase_amount
#     df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']  + df['auth_hist_card_id_size']
#     df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']+df['auth_hist_purchase_amount_sum']
#添加特征

for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    order_label1 = df_train.groupby([f])['outliers'].mean()
    df_train[f+'_outliers_mean'] = df_train[f].map(order_label1)
    df_test[f+'_outliers_mean'] = df_test[f].map(order_label1)
    
    order_label2 = df_train.groupby([f])['outliers'].sum()
    df_train[f+'_outliers_sum'] = df_train[f].map(order_label2)
    df_test[f+'_outliers_sum'] = df_test[f].map(order_label2)
    
#     order_label1 = df_train.groupby([f])['target'].mean()
#     df_train[f+'_target_mean'] = df_train[f].map(order_label1) 
#     df_test[f+'_target_sum'] = df_test[f].map(order_label1)
#     order_label2 = df_train.gorupby([f])['target'].sum()
#     df_train[f+'_target_sum'] = df_train[f].map(order_label2)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label2)
 
# get_dummies 似乎有一点点不良影响
df_train = pd.get_dummies(df_train,columns =['feature_1','feature_2'])
df_test = pd.get_dummies(df_test,columns =['feature_1','feature_2'])

In [29]:
#首次购买的时间居然早于首次激活的时间，进行调整
df_train.loc[df_train['auth_hist_first_buy'] < 0, 'auth_hist_first_buy'] = -1
df_train.loc[df_train['hist_first_buy'] < 0, 'hist_first_buy'] = -1

## Read features from merchant, because of memory problems, we extract this features seperately.

In [37]:
df=pd.read_csv('../../ELO/merchants.csv')

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [33]:
# df_merchants = pd.read_pickle('../input/elomerchantfeatures/merchant_features.pickle')
# for df in [df_train, df_test]:
#     df = df.merge(df_merchants, on = 'card_id', how = 'left')
# df_train.head()

NameError: name 'df_merchants' is not defined

In [39]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']
# del df_train['target']

In [40]:
len(df_train_columns)

269

## Above is the same with wang, I call it wang's features

## train a model with outliers

In [41]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [42]:
from sklearn.model_selection import KFold,StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=4950)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    
    # cal the outlier ratio for each fold
    cur_fold= df_train.iloc[trn_idx]
    number_outliers = len(cur_fold[cur_fold['outliers'] == 1])
    total = len(cur_fold)
    print("fold %s %s %s" % (fold_,number_outliers, number_outliers / total))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 600)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

fold 0 1986 0.010928600907965332
Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 3.65119	valid_1's rmse: 3.70999
[200]	training's rmse: 3.56314	valid_1's rmse: 3.67614
[300]	training's rmse: 3.5084	valid_1's rmse: 3.66502
[400]	training's rmse: 3.46922	valid_1's rmse: 3.65881
[500]	training's rmse: 3.43617	valid_1's rmse: 3.65474
[600]	training's rmse: 3.40929	valid_1's rmse: 3.6531
[700]	training's rmse: 3.38638	valid_1's rmse: 3.65237
[800]	training's rmse: 3.36389	valid_1's rmse: 3.6516
[900]	training's rmse: 3.34424	valid_1's rmse: 3.65155
[1000]	training's rmse: 3.32375	valid_1's rmse: 3.65048
[1100]	training's rmse: 3.3053	valid_1's rmse: 3.65008
[1200]	training's rmse: 3.28758	valid_1's rmse: 3.6496
[1300]	training's rmse: 3.27057	valid_1's rmse: 3.64981
[1400]	training's rmse: 3.25379	valid_1's rmse: 3.64983
[1500]	training's rmse: 3.23826	valid_1's rmse: 3.65035
[1600]	training's rmse: 3.22167	valid_1's rmse: 3.64937
[1700]	training's rmse

[1700]	training's rmse: 3.19923	valid_1's rmse: 3.68295
[1800]	training's rmse: 3.18288	valid_1's rmse: 3.68333
[1900]	training's rmse: 3.16814	valid_1's rmse: 3.68358
[2000]	training's rmse: 3.15454	valid_1's rmse: 3.68429
Early stopping, best iteration is:
[1455]	training's rmse: 3.23687	valid_1's rmse: 3.68199
fold 6 1986 0.010928600907965332
Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 3.65556	valid_1's rmse: 3.68767
[200]	training's rmse: 3.56991	valid_1's rmse: 3.63553
[300]	training's rmse: 3.51812	valid_1's rmse: 3.61766
[400]	training's rmse: 3.47902	valid_1's rmse: 3.60634
[500]	training's rmse: 3.44717	valid_1's rmse: 3.60011
[600]	training's rmse: 3.4196	valid_1's rmse: 3.59561
[700]	training's rmse: 3.39553	valid_1's rmse: 3.5927
[800]	training's rmse: 3.3727	valid_1's rmse: 3.58985
[900]	training's rmse: 3.35251	valid_1's rmse: 3.58857
[1000]	training's rmse: 3.33275	valid_1's rmse: 3.58791
[1100]	training's rmse: 3.31424	valid_1's

3.6428849229128195

In [ ]:
### LB: 3.676
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)
single_pd = pd.DataFrame({
        'card_id': df_test.card_id,
        'target': predictions
})
single_pd.to_csv('wang_newest_with_purchase_journeys.csv', index=False)